# Shell Hackathon Solution 2018

We know that the total capacity of the installed charger in 2018 meets the demand of 2018 nearly exactly. So the optimal number of chargers to be build should be zero.

In [6]:
from math import sqrt
from itertools import product

import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [7]:
supply = pd.read_csv("../data/raw/exisiting_EV_infrastructure_2018.csv")
demand = pd.read_csv("../data/raw/Demand_History.csv")

In [8]:
slow_charger = 200
slow_costs = 1.0*600
fast_charger = 400
fast_costs = 1.5*600

A fast charger delivers twice the capacity but cost only 1.5 times the amount of a slow charger. Building fast chargers is therefor optimal.

In [9]:
# real capacities
real_capacities = (supply["existing_num_SCS"]*slow_charger + supply["existing_num_FCS"]*fast_charger).tolist()
# maximum theoretical capacities
capacities = (supply["total_parking_slots"]*fast_charger).tolist()
demands = demand["2018"].tolist()

In [10]:
sum(capacities) > sum(demands), sum(capacities), sum(demands), sum(real_capacities)

(True, 1000000, 361529.6365968905, 361600)

The maximal capacity of all parking slots is more than enough to fulfill the demand.

In [11]:
# facilities are parking slots
facilities = list(supply[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))
# customers are areas of electricity demand
customers = list(demand[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))

In [12]:
# Compute key parameters of MIP model formulation
num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

In [13]:
# This function determines the Euclidean distance between a facility and customer sites.
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [14]:
# Compute distance matrix
distance = {(c,f): compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

In [15]:
# maximum slots for chargers
slots = supply["total_parking_slots"].tolist()

# existing chargers
slow_slots = supply["existing_num_SCS"].tolist()
fast_slots = supply["existing_num_FCS"].tolist()

In [16]:
## For testing set number of slots to zero
# slow_slots = [0]*len(facilities)
# fast_slots = [0]*len(facilities)

In [17]:
solver = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

In [19]:
# Variables
assign = {}
for i,j in distance.keys(): 
    assign[(i,j)] = solver.NumVar(0,solver.infinity(),"Assign")
# Do not delete existing chargers (Constraint/Boundaries 2) 
slow = {}
for j in range(num_facilities):
    slow[j] = solver.IntVar(slow_slots[j], solver.infinity(), "Slow")
fast = {}
for j in range(num_facilities):
    fast[j] = solver.IntVar(fast_slots[j], solver.infinity(), "Fast")          

In [20]:
# Constraint 3 (slots)
for j in range(num_facilities):
    solver.Add(slow[j] + fast[j] <= slots[j])
# Constraint 5 (capacity constraints)
for j in range(num_facilities):
    solver.Add(sum(assign[(i,j)] for i in range(num_customers)) <= (slow[j]*200 + fast[j]*400))
# Constraint 6 (demand constraints)
for i in range(num_customers):
    solver.Add(sum(assign[(i,j)] for j in range(num_facilities)) == demands[i] )    

In [22]:
objective = solver.Objective()
# Building costs for chargers
for j in range(num_facilities):
    objective.SetCoefficient(slow[j], slow_costs)
for j in range(num_facilities):
    objective.SetCoefficient(fast[j], fast_costs)
# distance costs
for i in range(num_customers):
    for j in range(num_facilities):
        objective.SetCoefficient(assign[(i,j)],distance[(i,j)]) 
objective.SetMinimization()    

The Shell Hackathon cost function includes the MAE of the demand prediction as additional term. We know the exact demand for 2018, so this term is zero. For 2019/2020 we cannot compute the term, because we only have the predicted values, but not the exact ones.

In [23]:
status = solver.Solve()

In [24]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 2276576.673923479

Problem solved in 366545.000000 milliseconds
Problem solved in 14110 iterations
Problem solved in 1 branch-and-bound nodes


In [29]:
# display optimal values of decision variables
lines = []
for j in range(num_facilities):
    build_slow = slow[j].solution_value() - slow_slots[j]
    build_fast = fast[j].solution_value() - fast_slots[j]
    if build_slow > 0 or build_fast > 0: 
        lines.append(f"Build {slow[j].solution_value() - slow_slots[j]} slow charger  and {fast[j].solution_value() - fast_slots[j]} fast charger at location {j + 1}.")
for line in lines[:5]: print(line)
print("...")
for line in lines[-5:]: print(line)
    
print("Number of construction sites:", len(lines))          

Build 0.0 slow charger  and 6.0 fast charger at location 1.
Build 0.0 slow charger  and 8.0 fast charger at location 11.
Build 0.0 slow charger  and 8.0 fast charger at location 13.
Build 0.0 slow charger  and 5.0 fast charger at location 16.
Build 0.0 slow charger  and 3.0 fast charger at location 18.
...
Build 0.0 slow charger  and 10.0 fast charger at location 91.
Build 0.0 slow charger  and 10.0 fast charger at location 93.
Build 0.0 slow charger  and 3.0 fast charger at location 96.
Build 0.0 slow charger  and 7.0 fast charger at location 97.
Build 0.0 slow charger  and 1.0 fast charger at location 98.
Number of construction sites: 24


Surprisingly we find that according to our solver we should build additional chargers at 24 parking slots.

In [32]:
# compute the total electricity supply of our solution
total_supply = 0
for j in range(num_facilities):
    total_supply += slow[j].solution_value()*200 + fast[j].solution_value()*400

In [31]:
sum(demands), total_supply 

(361529.6365968905, 429600.0)

Our supply is much bigger that our demand. I don't understand why the solver comes up with that. Maybe this lowers the distance costs?

In [34]:
# compute the total demand of our solution
total_demand = 0.0
for i in range(num_customers):
    for j in range(num_facilities):
        total_demand += assign[(i,j)].solution_value()

In [35]:
sum(demands), total_demand

(361529.6365968905, 361529.6365968906)

The total demand of our solution is exactly the sum of input demands. So we don't violate that constraint. 

In [27]:
# Shipments from facilities to customers.
lines = []
for i, j in assign.keys():
    if (abs(assign[i, j].solution_value()) > 1e-6):
        lines.append(f"Demand point {i + 1} receives {round(assign[i, j].solution_value(), 2)} of its demand {round(demands[i],2)} from parking slot {j + 1} .")
for line in lines[:5]: print(line)
print("...")
for line in lines[-5:]: print(line)        

Demand point 1 receives 13.12 of its demand 13.12 from parking slot 39 .
Demand point 2 receives 12.02 of its demand 12.02 from parking slot 39 .
Demand point 3 receives 14.02 of its demand 14.02 from parking slot 39 .
Demand point 4 receives 15.01 of its demand 15.01 from parking slot 39 .
Demand point 5 receives 16.36 of its demand 16.36 from parking slot 39 .
...
Demand point 4092 receives 5.43 of its demand 5.43 from parking slot 17 .
Demand point 4093 receives 2.06 of its demand 2.06 from parking slot 17 .
Demand point 4094 receives 3.22 of its demand 3.22 from parking slot 17 .
Demand point 4095 receives 6.26 of its demand 6.26 from parking slot 17 .
Demand point 4096 receives 6.86 of its demand 6.86 from parking slot 17 .
